# Assiment 10

#### Get the 15 variables from this raster for all Peru departments polygons. This is the link where shapefiles are located. This is the link of the source raster. The values should be the percentage of district area cover by this specific Morphological Settlement Zone.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install rasterio

In [ ]:
# requirementss
%pip install geopandas matplotlib shapely rasterio numpy pandas sklearn-xarray -q
%pip install git+https://github.com/jgrss/geowombat  -q

In [ ]:
%pip install rasterstats

In [ ]:
!pip install geopandas

In [ ]:
import geopandas as gpd
from rasterstats import zonal_stats
import pandas as pd
import geowombat as gw

import rasterio
import matplotlib.pyplot as plt
from rasterio.merge import merge
from rasterio.plot import show
from rasterio.mask import mask

import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Point, LineString
import numpy as np
import chardet

import pyproj
from shapely.ops import transform

from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe

In [ ]:
shapefile_path = '/content/drive/MyDrive/TIF/INEI_LIMITE_DEPARTAMENTAL.shp'
department = gpd.read_file(shapefile_path)
print(department.head())

In [ ]:
transformer = pyproj.Transformer.from_crs('epsg:4326', 'esri:54009', always_xy=True)

# Define a function to apply the transformation
def apply_transform(geom):
    return transform(transformer.transform, geom)

# Apply the transformation to the geometries
department['geometry'] = department['geometry'].apply(apply_transform)

In [ ]:
# List of raster file paths
tif_file_path = [
    '/content/drive/MyDrive/TIF/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C10.tif',
    '/content/drive/MyDrive/TIF/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C11.tif',
    '/content/drive/MyDrive/TIF/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C12.tif',
    '/content/drive/MyDrive/TIF/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R11_C11.tif',
    '/content/drive/MyDrive/TIF/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R11_C12.tif',
    '/content/drive/MyDrive/TIF/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R12_C11.tif',
    '/content/drive/MyDrive/TIF/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R12_C12.tif',
    '/content/drive/MyDrive/TIF/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R9_C10.tif',
    '/content/drive/MyDrive/TIF/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R9_C11.tif',
    '/content/drive/MyDrive/TIF/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R9_C12.tif'
]

In [ ]:
# We generate a list to obtain the raster data
raster_data = []

# Iterate over each raster file
for raster_path in tif_file_path:
    # We intercept the raster information with the shp file
    stats = zonal_stats(department, raster_path, stats="count sum", categorical=True, all_touched=True)
    # Convert Statistics to DataFrame
    stats_df = pd.DataFrame(stats)
    # In the created list we concatenate the geometry and raster information of the departments
    df = pd.concat([department.reset_index(drop=True), stats_df], axis=1)
    raster_data.append(df)

In [ ]:
# Concatenar todos los DataFrames en all_stats en un único DataFrame
final_df_raster = pd.concat(raster_data, ignore_index=True)

# Verificar el resultado
print(final_df_raster.head())